# Entraînement Local du Modèle Morningstar

In [ ]:
# Installer les dépendances si nécessaire
%pip install tensorflow pandas numpy

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from pathlib import Path
import sys
import os

# Définir le chemin absolu du projet
PROJECT_ROOT = Path(os.getcwd()).parent

# Ajouter le répertoire racine au PYTHONPATH
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

from model.training.data_loader import load_and_split_data
from model.architecture.morningstar_model import MorningstarModel

In [ ]:
# Configuration
DATA_PATH = str(PROJECT_ROOT / 'data' / 'processed' / 'temp_sol_training_local.parquet')
MODEL_SAVE_PATH = str(PROJECT_ROOT / 'model' / 'training' / 'morningstar_local.h5')

# Mapping complet pour les régimes de marché
REGIME_MAPPING = {
    'bearish': 0,
    'bearish_placeholder': 0,
    'sideways': 1,
    'sideways_placeholder': 1,
    'bullish': 2,
    'bullish_placeholder': 2
}

In [ ]:
# Chargement des données
(x_train, x_llm_train), y_train = load_and_split_data(
    DATA_PATH,
    label_columns=['trading_signal', 'volatility', 'market_regime', 'level_sl', 'level_tp'],
    label_mappings={'market_regime': REGIME_MAPPING},
    as_tensor=True
)

print(f"Données chargées - X_train: {x_train.shape}, X_llm: {x_llm_train.shape}")
print(f"Labels disponibles: {list(y_train.keys())}")

In [ ]:
# Initialisation du modèle
model = MorningstarModel()
model.initialize_model()

# Configuration de compilation
losses = {
    'signal': 'sparse_categorical_crossentropy',
    'volatility_quantiles': 'mse',
    'market_regime': 'sparse_categorical_crossentropy',
    'sl_tp': 'mse'
}

metrics = {
    'signal': ['accuracy'],
    'volatility_quantiles': ['mae'],
    'market_regime': ['accuracy'],
    'sl_tp': ['mae']
}

model.model.compile(
    optimizer='adam',
    loss=losses,
    metrics=metrics
)

In [ ]:
# Entraînement
history = model.model.fit(
    {'technical_input': x_train, 'llm_input': x_llm_train},
    {'signal': y_train['trading_signal'], 
     'volatility_quantiles': y_train['volatility'],
     'market_regime': y_train['market_regime'],
     'sl_tp': tf.stack([y_train['level_sl'], y_train['level_tp']], axis=1)},
    epochs=50,
    batch_size=32,
    validation_split=0.2
)

In [ ]:
# Sauvegarde
Path(MODEL_SAVE_PATH).parent.mkdir(parents=True, exist_ok=True)
model.model.save(MODEL_SAVE_PATH)
print(f"Modèle sauvegardé à {MODEL_SAVE_PATH}")